# COVID-19 emerging reseaerch

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import json
from re import search
import re

In [ ]:
data = pd.read_csv("COVID19 - COVID-19.csv")

In [ ]:
def get_page(URL):
    """
    get the page of a given journal article
    """
    headers = {
        "User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'
    }
    try:
        resp = requests.get(URL, headers = headers)
        if resp.status_code == 200:
            return bs(resp.text)
        else:
            print(resp.status_code)
    except:
        print("Page retrieval failed")

def get_author_address(soup):
    """
    parse first author address from paper metadata
    """
    try:
        preout = soup.findAll("meta", {"name": "citation_author_institution"})
        for x in preout:
            if len(x.get("content")) != 0:
                out = x.get("content")
                break
        return out
    except:
        try:
            out = soup.find("p", {"class": "affiliation"}).text
            return out
        except:
            try:
                out = soup.find("div", {"class": "accordion article__author-affiliations"}).text
                return out
            except:
                try:
                    out = soup.find("span", {"class": "aff-overlay"}).findAll("li")[1].text
                    return out
                except:
                    try:
                        out = soup.find("span", {"class": "affiliation"}).text
                        return out
                    except:
                        try:
                            out = soup.find("div", {"class": "artice-info-affiliation"}).text
                            return out
                        except:
                            try:
                                out = soup.find("div", {"class": "author-info"}).text
                                return out
                            except:
                                try:
                                    out = soup.find("div", {"class": "loa-info-affiliations-info"}).text
                                    return out
                                except:
                                    try:
                                        out = soup.find("div", {"class": "affiliation-name"}).text
                                        return out
                                    except:
                                        try:
                                            preout = soup.findAll("p", {"class": "f-body"})
                                            for x in preout:
                                                if x.find("a", {"class": "email"}):
                                                    out = x.text
                                                    break
                                            return out
                                        except:
                                            try:
                                                out = soup.find("section", {"id": "author_affiliations"}).find("p", {"class": "f-body--sm"}).text
                                                return out
                                            except:
                                                try:
                                                    out = soup.find("td", {"class": "PubAff"}).text
                                                    return out
                                                except:
                                                    try:
                                                        out = soup.find("div", {"class": "origin_section02"}).findAll("p")[1].text
                                                        return out
                                                    except:
                                                        try:
                                                            out = soup.find("div", {"class": "article-inner_html"}).findAll("p")
                                                            return out
                                                        except:
                                                            try:
                                                                out = soup.find("a", {"class": "entryAuthor"}).find("span", {"class": "overlay"}).text
                                                                return out
                                                            except:
                                                                try:
                                                                    out = soup.find("div", {"class": "aff"}).text
                                                                    return out
                                                                except:
                                                                    try:
                                                                        out = soup.find("span", {"id": re.compile(r"affilia\d+")}).text
                                                                        return out
                                                                    except:
                                                                        try:
                                                                            out = soup.find("p", {"id": re.compile(r"AF_ID.*?")}).text
                                                                            return out
                                                                        except:
                                                                            try:
                                                                                out = soup.find("span", {"class": "affiliations"}).text
                                                                                return out
                                                                            except:
                                                                                try:
                                                                                    out = soup.find("dl", {"id": "afflist1"}).find("dt").text
                                                                                    return out
                                                                                except:
                                                                                    print("Address not found")

In [ ]:
no_address_data = data[data["Full Address"].str.contains("^\s+$")]
urls = no_address_data["URL"].tolist()

In [ ]:
pages = list()

In [ ]:
done_url = [x["url"] for x in pages]
for url in urls:
    if url is not np.nan and url not in done_url:
        print(url)
        soup = get_page(url)
        pages.append(
            {
                "url": url,
                "page": soup
            }
        )


In [ ]:
len(pages)

In [ ]:
addresses = list()

In [ ]:
for page in pages:
    author_address = get_author_address(page["page"])
    addresses.append(
        {
            "url": page["url"],
            "full_address": author_address
        }
    )

In [ ]:
missing = [x["url"] for x in addresses if not x["full_address"]]
len(missing)

In [ ]:
excluded = ["http://dx.doi.org/10.33321/cdi.2020.44.21", "http://dx.doi.org/10.4062/biomolther.2019.202", "http://dx.doi.org/10.3346/jkms.2020.35.e79", "http://dx.doi.org/10.1001/jama.2020.2633", "http://dx.doi.org/10.3348/kjr.2020.0112", "http://dx.doi.org/10.1056/NEJMe2004244", "http://dx.doi.org/10.2471/blt.20.251561", "http://dx.doi.org/10.26355/eurrev_202002_20396", "http://dx.doi.org/10.24875/RIC.20000023", "http://dx.doi.org/10.1021/cen-09809-leadcon", "http://dx.doi.org/10.1136/vr.m1009", "http://dx.doi.org/10.1111/ijcp.13365", "http://dx.doi.org/10.1002/adaw.32661", "http://dx.doi.org/10.1017/S1049023X2000031X", "http://dx.doi.org/10.21037/apm.2020.02.33", "http://dx.doi.org/10.1056/NEJMe2004856", "http://dx.doi.org/10.1701/3315.32860", "http://dx.doi.org/10.26434/CHEMRXIV.11902623.V2", "http://dx.doi.org/10.1111/tan.13841"]
unparsed_pages = [x for x in pages if x["url"] in missing and x["page"] and "ipAddress : '188.29.188.131'" not in x["page"].get_text() and not search(r"\/science\.", x["url"]) and "Thank you for visiting nature.com" not in x["page"].get_text() and not x["page"].find("a", {"href": "mailto:NewMedia@cma.org.cn"}) and x["url"] not in excluded and "CHEMRXIV" not in x["url"] and not search(r"\/cen\-\d+\-", x["url"]) and not search(r"\/vr\.[a-z0-9]+$", x["url"])]

In [ ]:
len(unparsed_pages)

In [ ]:
unparsed_pages[0]["page"]#.find("dl", {"id": "afflist1"}).find("dt").text
# <span id="sp_affiliation">  </span>

In [ ]:
unparsed_pages[0]["url"]

In [ ]:
parsed_addr = [x for x in addresses if x["full_address"]]
len(parsed_addr)

In [ ]:
df = pd.DataFrame(parsed_addr)

In [ ]:
df.to_csv("article_addresses.csv", index = False)